# Tutorial 1: Docker Basics and LightyNode Setup

## Prerequisites
- Docker installed and running
- Python 3.8 or higher
- Basic understanding of command line operations

## Step 1: Virtual Environment Setup

**IMPORTANT: Set up a Python virtual environment first!**

Before running the notebooks, create and activate a virtual environment with required packages. All dependencies are listed in the `requirements.txt` file for easy installation.

In [1]:
# Verify installation and import common utilities
try:
    import docker
    import requests
    import urllib3
    from common_utils import StatusIndicators, print_status, DockerHelper
    print_status("All required packages installed", StatusIndicators.SUCCESS)
except ImportError as e:
    print_status(f"Missing package: {e}", StatusIndicators.ERROR)
    print("Install with: pip install -r requirements.txt")

[SUCCESS] All required packages installed


## Step 2: Student Configuration

**IMPORTANT: Change the STUDENT_ID to your assigned number!**

Each student must use a unique ID to create isolated environments.

In [ ]:
# === STUDENT CONFIGURATION ===
# CHANGE THIS TO YOUR ASSIGNED STUDENT ID!
STUDENT_ID = "1"  # Example: "1", "2", "3", etc.

# Generated configuration (DO NOT CHANGE)
NETWORK_NAME = f"student{STUDENT_ID}-network"
print(f"Student {STUDENT_ID} environment configured")
print(f"Network: {NETWORK_NAME}")

## Step 3: Import Required Libraries

We'll use Docker Python SDK to manage containers programmatically.

In [ ]:
import docker
import time
from docker.errors import NotFound, APIError
from common_utils import StatusIndicators, print_status, DockerHelper

# Initialize Docker helper
docker_helper = DockerHelper()

## Step 4: Helper Functions

These functions help manage student-specific resources.

In [ ]:
# Helper functions

def get_container_name(base_name):
    """Generate student-specific container name"""
    return f"student{STUDENT_ID}-{base_name}"

def create_network():
    """Create isolated Docker network for this student"""
    try:
        network = docker_helper.client.networks.get(NETWORK_NAME)
        print_status(f"Network {NETWORK_NAME} already exists", StatusIndicators.INFO)
        return network
    except docker.errors.NotFound:
        network = docker_helper.client.networks.create(NETWORK_NAME, driver="bridge")
        print_status(f"Created network: {NETWORK_NAME}", StatusIndicators.SUCCESS)
        return network

def start_lightynode(node_name, device_type, image=None):
    """Start a LightyNode container"""
    container_name = get_container_name(node_name)

    # Select appropriate image based on device type
    if image is None:
        if 'roadm' in device_type.lower():
            image = "aparaajitha/lightynode-roadma-221"
        elif 'tpdr' in device_type.lower() or 'xpdr' in device_type.lower():
            image = "aparaajitha/lightynode-xpdra2-71"
        else:
            raise ValueError(f"Unknown device type: {device_type}")

    try:
        container = docker_helper.client.containers.get(container_name)
        if container.status == "running":
            print_status(f"Container {container_name} already running", StatusIndicators.INFO)
            return container
        else:
            container.start()
            print_status(f"Started existing container {container_name}", StatusIndicators.SUCCESS)
            return container
    except docker.errors.NotFound:
        container = docker_helper.client.containers.run(
            image=image,
            name=container_name,
            network=NETWORK_NAME,
            detach=True,
            remove=False
        )
        print_status(f"Created container: {container_name}", StatusIndicators.SUCCESS)
        return container

def wait_for_lightynode(container, timeout=60):
    """Wait for LightyNode to be ready"""
    ready_messages = [b"Data tree change listeners registered"]
    return docker_helper.wait_for_container(container.name, ready_messages, timeout)

def show_container_status():
    """Display status of student's containers"""
    all_containers = docker_helper.client.containers.list(all=True)
    student_containers = [c for c in all_containers if c.name.startswith(f"student{STUDENT_ID}-")]

    if not student_containers:
        print_status("No containers found for this student", StatusIndicators.INFO)
        return

    print(f"Student {STUDENT_ID} containers:")
    for container in student_containers:
        container.reload()
        image_name = container.image.tags[0] if container.image.tags else 'None'
        print(f"  {container.name}: {container.status} (image: {image_name})")

print_status("Helper functions defined", StatusIndicators.SUCCESS)

## Step 5: Create Isolated Network

Each student gets their own Docker network to prevent interference with other students.

In [ ]:
# Create student's isolated network
network = create_network()

# Display network information (simplified)
try:
    network_info = docker_helper.client.api.inspect_network(network.id)
    ipam_config = network_info.get('IPAM', {}).get('Config', [])
    if ipam_config and len(ipam_config) > 0:
        config = ipam_config[0]
        subnet = config.get('Subnet', 'Auto-assigned')
        gateway = config.get('Gateway', 'Auto-assigned')
    else:
        subnet = 'Auto-assigned'
        gateway = 'Auto-assigned'

    print_status(f"Network Details", StatusIndicators.INFO,
                f"Name: {network_info['Name']}, Driver: {network_info['Driver']}")
    print_status(f"Subnet: {subnet}, Gateway: {gateway}", StatusIndicators.INFO)

except Exception as e:
    print_status("Error getting network details", StatusIndicators.WARNING,
                "Network created successfully but could not retrieve details")

## Step 6: Pull LightyNode Images

Let's pull the required Docker images from Docker Hub.

In [ ]:
# Pull required Docker images
images_to_pull = [
    "aparaajitha/lightynode-roadma-221",
    "aparaajitha/lightynode-xpdra2-71"
]

print("Pulling LightyNode Docker images...")

for image in images_to_pull:
    try:
        docker_helper.client.images.pull(image)
        print_status(f"Pulled {image}", StatusIndicators.SUCCESS)
    except Exception as e:
        print_status(f"Failed to pull {image}", StatusIndicators.ERROR)
        print(f"Error: {e}")

# List available images
lightynode_images = [img for img in docker_helper.client.images.list()
                    if any(tag.startswith('aparaajitha/lightynode')
                          for tag in img.tags) if img.tags]

print(f"Available LightyNode images: {len(lightynode_images)}")
for img in lightynode_images:
    for tag in img.tags:
        if tag.startswith('aparaajitha/lightynode'):
            print(f"  {tag}")

## Step 7: Start ROADM Device

We'll start with a ROADM (Reconfigurable Optical Add-Drop Multiplexer) device.

In [ ]:
# Start ROADM device
print("Starting ROADM device...")
ROADM_NAME = "roadm-a"
roadm_container = start_lightynode(ROADM_NAME, "roadm")

# Wait for container to be ready
if wait_for_lightynode(roadm_container):
    print_status(f"ROADM container ready", StatusIndicators.SUCCESS)
else:
    print_status(f"ROADM container failed to start", StatusIndicators.ERROR)

## Step 8: Start Transponder Device

Let's also start a transponder device for complete network simulation.

In [ ]:
# Start transponder device
print("Starting transponder device...")
TPDR_NAME = "tpdr-a"
tpdr_container = start_lightynode(TPDR_NAME, "xpdr")

# Wait for container to be ready
if wait_for_lightynode(tpdr_container):
    print_status(f"Transponder container ready", StatusIndicators.SUCCESS)
else:
    print_status(f"Transponder container failed to start", StatusIndicators.ERROR)

## Step 9: Verify Container Status

Let's check that our containers are running properly.

In [ ]:
show_container_status()

## Step 10: Test Network Connectivity

Verify that containers can communicate within the isolated network.

In [ ]:
def test_network_connectivity():
    """Test network connectivity between containers"""
    roadm_name = get_container_name(ROADM_NAME)
    tpdr_name = get_container_name(TPDR_NAME)

    try:
        roadm = docker_helper.client.containers.get(roadm_name)
        result = roadm.exec_run(f"ping -c 2 {tpdr_name}")

        if result.exit_code == 0:
            print_status(f"{roadm_name} can ping {tpdr_name}", StatusIndicators.SUCCESS)
        else:
            print_status(f"{roadm_name} cannot ping {tpdr_name}", StatusIndicators.ERROR)
    except Exception as e:
        print_status("Error testing connectivity", StatusIndicators.ERROR)
        print(f"Error: {e}")

test_network_connectivity()

## Step 11: View Container Logs

Let's examine the logs to understand what's happening inside our containers.

In [ ]:
def show_container_logs(container_name, lines=5):
    """Display last few lines of container logs (simplified)"""
    try:
        container = docker_helper.client.containers.get(container_name)
        logs = container.logs(tail=lines).decode('utf-8')

        print_status(f"Last {lines} lines from {container_name}", StatusIndicators.INFO)
        for line in logs.split('\n'):
            if line.strip():
                # Only show meaningful log lines, truncate very long ones
                log_line = line.strip()
                if len(log_line) > 1000:
                    log_line = log_line[:1000]
                print_status(log_line, StatusIndicators.INFO)

    except Exception as e:
        print_status(f"Error getting logs for {container_name}", StatusIndicators.ERROR, str(e))

# Show logs for both containers (simplified)
show_container_logs(get_container_name(ROADM_NAME), lines=3)
show_container_logs(get_container_name(TPDR_NAME), lines=3)

## Summary

In this tutorial, you have:

1. **Set up a virtual environment** with required Python packages
2. **Created an isolated environment** using Docker networks
3. **Pulled LightyNode images** from Docker Hub
4. **Started LightyNode containers** (ROADM and Transponder)
5. **Verified container status** and network connectivity
6. **Tested basic connectivity** to the containers

Your containers are now ready for SDN operations.

## Next Steps

In the next tutorial, we'll learn how to:
- Set up TransportPCE SDN controller
- Connect devices to the controller
- Use RESTCONF for device management

## Troubleshooting

If you encounter issues:

1. **Check Docker is running**: `docker ps`
2. **Verify network exists**: `docker network ls | grep student{STUDENT_ID}`
3. **Check container logs**: Use the `show_container_logs()` function above
4. **Restart containers**: Stop and remove containers, then run this notebook again
5. **Virtual environment**: Ensure you activated your virtual environment and installed packages
   ```bash
   source lightynode-tutorial/bin/activate
   pip install -r requirements.txt
   ```

## Requirements

All Python dependencies are specified in `requirements.txt`:
- `docker>=6.0.0` - Container management
- `requests>=2.31.0` - HTTP operations for RESTCONF
- `urllib3>=2.0.0` - SSL/TLS handling

## Cleanup (Optional)

To clean up your environment (warning: this will stop and remove all your containers):

In [ ]:
# Cleanup function (uncomment to use)
def cleanup_student_environment():
    """Stop and remove all student containers and network"""
    all_containers = docker_helper.client.containers.list(all=True)
    student_containers = [c for c in all_containers if c.name.startswith(f"student{STUDENT_ID}-")]

    for container in student_containers:
        try:
            container.stop(timeout=10)
            container.remove()
            print_status(f"Removed {container.name}", StatusIndicators.SUCCESS)
        except Exception as e:
            print_status(f"Error removing {container.name}", StatusIndicators.ERROR)
            print(f"Error: {e}")

    # Remove network
    try:
        network = docker_helper.client.networks.get(NETWORK_NAME)
        network.remove()
        print_status(f"Removed network: {NETWORK_NAME}", StatusIndicators.SUCCESS)
    except Exception as e:
        print_status(f"Error removing network", StatusIndicators.ERROR)
        print(f"Error: {e}")

# To run cleanup, uncomment the line below:
# cleanup_student_environment()